In [1]:
import torch

In [4]:
features = torch.rand((5, 3, 5, 3))

In [ ]:
features

In [17]:
def get_normalize(features: torch.Tensor):
    mean = features.mean(dim=(0, 2, 3))
    std = features.std(dim=(0, 2, 3))
    return mean, std

In [18]:
get_normalize(features)

(tensor([0.5264, 0.4851, 0.5517]), tensor([0.2625, 0.3126, 0.3157]))

In [20]:
import torchvision.transforms as T

In [28]:
def get_augmentations(train: bool = True) -> T.Compose:
    trans = []
    trans.append(T.Resize((224, 224)))
    if train:
        trans.append(T.RandomHorizontalFlip(p=0.5))
    trans.append(T.ToTensor())
    trans.append(T.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]))
    transforms = T.Compose(trans)
    return transforms

In [30]:
get_augmentations(True)

Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=None)
    RandomHorizontalFlip(p=0.5)
    ToTensor()
    Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
)

In [35]:
import torch.nn as nn
from torch.utils.data import DataLoader

In [40]:
@torch.inference_mode()
def predict(model: nn.Module, loader: DataLoader, device: torch.device):
    model = model.to(device)
    predictions = []
    model.eval()
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        output = model(inputs)
        _, predicted_classes = torch.max(output, dim=1)
        predictions.append(predicted_classes.cpu())
        
    return torch.cat(predictions)

In [39]:
# predict(nn.Module(), DataLoader(), torch.device())

TypeError: __init__() missing 1 required positional argument: 'dataset'

In [41]:
def predict_tta(model: nn.Module, loader: DataLoader, device: torch.device, iterations: int = 2):
    model = model.to(device)
    predictions = []
    model.eval()
    
    for i in range(iterations):
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            output = model(inputs)
            _, predicted_classes = torch.max(output, dim=1)